###### This notebook parses the HEDIS NCQA into patient panels by Organization 
##### Added Excel Automation including Filtering, Styling, Security, Pre-filtering 
#### Joe Eberle - 4/26/2022  


In [ ]:
## -- Current Treated Lives 
## select * from DS_ATTRIB_AFFILIA_PATIENT_DETAIL -- For attribution 
## select * from DS_TREATED_LIVES   -- For Treated Lives 
## select  TL.ORGANIZATION_CLASS, TL.ORGANIZATION_NAME, TL.PROVIDER_NAME, TL.PATIENT_ID from DS_TREATED_PATIENT_DETAIL TL

In [1]:
# Import Libraries for Spreadsheet Automation 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import random
import math
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle, Font, Border, Side, PatternFill, GradientFill, Alignment, Color, Fill
from openpyxl import Workbook
import openpyxl
import time
from openpyxl.utils.dataframe import dataframe_to_rows 
from openpyxl.formatting.rule import DataBarRule
from openpyxl.formatting.rule import ColorScaleRule



In [2]:
# Configure the code and visual settings to have consistency 
LIGHT_BLUE = "C5D9F1"
YELLOW = "FFFF00"
GRAY = "E4DFEC"
DEEP_BLUE = "1072BA"
WHITE = "FFFFFF"

CONTENT_TAB_COLOR = DEEP_BLUE
USER_TAB_COLOR = YELLOW
BLANK_WHITE = WHITE 
HEADER_COLOR = LIGHT_BLUE  # Light Blue - Light Blue indicates relevant content for validation 
HEADER_COLOR_USER_ENTRY = YELLOW # Yellow - Yellow indicates a field for User Entry 
HEADER_COLOR_INFORMATIONAL_ONLY = GRAY # Gray - Indicates a field that maybe helpful but for information purposes only 

Color_Scale_Rule = ColorScaleRule(start_type="min",start_color="FF0000",end_type="max",end_color="99FF33")
Data_Bar_Rule = DataBarRule(start_type="min", end_type="max", color="00CC00")

In [3]:
# Read in the CSV SQL output for Patient Panel
path = 'c:/Users/josep/Documents/Kaleida/Quality/'
Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/Quality/'
Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/Quality_ft/'
# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'
Path_to_Raw_Data = 'c:/Users/josep/Documents/Kaleida/'

Path_to_Raw_Patient_CSV = "C:/Users/josep/Documents/Kaleida/NCQA HEDIS Patient.csv"

Raw_Data_File_Name_1 = 'NCQA HEDIS Quality Measures by Patient Treated Lives.xlsx'
Raw_Data_File_Name_2 = 'NCQA HEDIS Quality Measures by Provider Treated Lives.xlsx'
Raw_Data_File_Name_3 = 'NCQA HEDIS Quality Measures by Organization Treated Lives.xlsx'
Raw_Data_File_Name_4 = 'NCQA HEDIS Quality Measures by Organization Class Treated Lives.xlsx' 


#"C:\Users\josep\Documents\Kaleida\NCQA HEDIS Quality Measures by Provider Treated Lives.xlsx"
#"C:\Users\josep\Documents\Kaleida\NCQA HEDIS Quality Measures by Organization Treated Lives.xlsx"
#"C:\Users\josep\Documents\Kaleida\NCQA HEDIS Quality Measures by Patient Treated Lives.xlsx"
#"C:\Users\josep\Documents\Kaleida\NCQA HEDIS Quality Measures by Organization Class Treated Lives.xlsx"




c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Patient Treated Lives.xlsx
c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Provider Treated Lives.xlsx
c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Organization Treated Lives.xlsx
c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Organization Class Treated Lives.xlsx


In [20]:
# Read in the CSV SQL output for Patient Panel
# patient_panel  = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_1)
patient_panel  = pd.read_csv("C:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality by Patient Treated Lives.csv")
provider_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_2)
organization_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_3)
organization_class_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_4)

#print(Path_to_Raw_Data + Raw_Data_File_Name_1)
print("C:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality by Patient Treated Lives.csv") # raw CSV limit 100000 ???? 
print(Path_to_Raw_Data + Raw_Data_File_Name_2)
print(Path_to_Raw_Data + Raw_Data_File_Name_3)
print(Path_to_Raw_Data + Raw_Data_File_Name_4)

C:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality by Patient Treated Lives.csv
c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Provider Treated Lives.xlsx
c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Organization Treated Lives.xlsx
c:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality Measures by Organization Class Treated Lives.xlsx


In [17]:
patient_panel  = pd.read_csv("C:/Users/josep/Documents/Kaleida/NCQA HEDIS Quality by Patient Treated Lives.csv")


In [18]:
patient_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   Organization_Class       1000000 non-null  object
 1   Organization_Name        1000000 non-null  object
 2   Provider_Name            1000000 non-null  object
 3   Patient_Name             1000000 non-null  object
 4   Measure_Qualified_Name   1000000 non-null  object
 5   Measure_Name_Simplified  1000000 non-null  object
 6   Total_Measure_Count      1000000 non-null  int64 
 7   Total_Not_Met_Count      1000000 non-null  int64 
 8   Total_Met_Count          1000000 non-null  int64 
 9   Birth_Date               1000000 non-null  object
 10  Gender                   531049 non-null   object
 11  Phone_Number             998651 non-null   object
 12  Patient_EMPI_ID          1000000 non-null  object
dtypes: int64(3), object(10)
memory usage: 99.2+ MB


In [21]:
# Optional Verify the input file structures 
#patient_panel.info()
#Organization_Provider_Summary.head()
organization_class_panel.columns

Index(['Organization_Class', 'Measure_Qualified_Name',
       'Measure_Name_Simplified', 'Total_Measure_Count', 'Total_Not_Met_Count',
       'Total_Met_Count'],
      dtype='object')

In [5]:
# Filter to Measures that are analytics only , calculate Percent complete and round 
#organization_class_panel = organization_class_panel[(organization_class_panel.Is_Registries_Measure == 'Analytics Only')]   
#organization_panel = organization_panel[(organization_panel.Is_Registries_Measure == 'Analytics Only')]   
#provider_panel = provider_panel[(provider_panel.Is_Registries_Measure == 'Analytics Only')]   
#patient_panel = patient_panel[(patient_panel.Is_Registries_Measure == 'Analytics Only')]  


In [22]:
# Filter to Measures that are analytics only , calculate Percent complete and round 
organization_class_panel['Measure_Completion_Percentage'] = ((organization_class_panel['Total_Met_Count'] / organization_class_panel['Total_Measure_Count']) * 100.0) 
organization_class_panel['Measure_Completion_Percentage'] = organization_class_panel['Measure_Completion_Percentage'].round(2)

organization_panel['Measure_Completion_Percentage'] = ((organization_panel['Total_Met_Count'] / organization_panel['Total_Measure_Count']) * 100.0) 
organization_panel['Measure_Completion_Percentage'] = organization_panel['Measure_Completion_Percentage'].round(2)

provider_panel['Measure_Completion_Percentage'] = ((provider_panel['Total_Met_Count'] / provider_panel['Total_Measure_Count']) * 100.0) 
provider_panel['Measure_Completion_Percentage'] = provider_panel['Measure_Completion_Percentage'].round(2)

patient_panel['Measure_Completion_Percentage'] = ((patient_panel['Total_Met_Count'] / patient_panel['Total_Measure_Count']) * 100.0) 
patient_panel['Measure_Completion_Percentage'] = patient_panel['Measure_Completion_Percentage'].round(2)


In [26]:
# simplify and Organize the organization quality Panels 
organization_panel = organization_panel[['Measure_Name_Simplified', 'Total_Measure_Count',
       'Total_Not_Met_Count', 'Total_Met_Count', 'Measure_Completion_Percentage',   'Organization_Class', 'Organization_Name']]
organization_class_panel = organization_class_panel[['Measure_Name_Simplified', 'Total_Measure_Count',
       'Total_Not_Met_Count', 'Total_Met_Count', 'Measure_Completion_Percentage',   'Organization_Class']]
provider_panel = provider_panel[['Provider_Name','Measure_Name_Simplified', 'Total_Measure_Count',
       'Total_Not_Met_Count', 'Total_Met_Count', 'Measure_Completion_Percentage',   'Organization_Class', 'Organization_Name']]
patient_panel = patient_panel[['Patient_Name','Provider_Name','Measure_Name_Simplified', 'Total_Measure_Count',
       'Total_Not_Met_Count', 'Total_Met_Count', 'Measure_Completion_Percentage',  'Organization_Class', 'Organization_Name', 'Birth_Date',
       'Gender','Phone_Number','Patient_EMPI_ID']]



In [27]:
patient_panel.columns

Index(['Patient_Name', 'Provider_Name', 'Measure_Name_Simplified',
       'Total_Measure_Count', 'Total_Not_Met_Count', 'Total_Met_Count',
       'Measure_Completion_Percentage', 'Organization_Class',
       'Organization_Name', 'Birth_Date', 'Gender', 'Phone_Number',
       'Patient_EMPI_ID'],
      dtype='object')

In [30]:
# The following function creates patient panels based upon organization name 
def create_organization_patient_panel(org_name):
    Organization_panel = patient_panel[(patient_panel.Organization_Name == org_name)]    
    patient_rows = len(Organization_panel) - 1 
    if patient_rows > 2:
        spread_name = Organization_panel['Organization_Name'].iloc[0]
        if spread_name == '':
            spread_name = 'No_ORG_NAME_'  
        spread_name = spread_name.replace('/','')  
      #  lesser = min(patient_rows,25)
      #  for x in range(lesser):
      #      Organization_panel.iat[random.randint(0,patient_rows),11] = 'Y'
       # spath = 'c:/Users/josep/Documents/Kaleida/Validation/'
        spath = Path_to_Excel
        spread_name = spath + spread_name.replace(' ','_') + '.xlsx'
        spread_name = spread_name.replace(' ','')
        Organization_panel.to_excel(spread_name,sheet_name='Patient Panel', index = False)  
        print(spread_name)
        return org_name

In [31]:
Org_names = patient_panel.Organization_Name.unique()
for org in Org_names:
    create_organization_patient_panel(org)

c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Amherst_Sheridan.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Buffalo_Main.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Dunkirk_Vineyard_Ste_1.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_East_Aurora_Main.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Elma_Transit.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Hamburg_Camp.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Hamburg_Southwestern.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Lancaster_Transit.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Lockport_Olcott.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Lockport_Snyder.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Medina_Ohio.xlsx
c:/Users/josep/Documents/Kaleida/Quality/GPPC_PCP_Orchard_Park_Big_Tree_Ste_107.xlsx


In [32]:
# Add the Provider Summary Excel Tab  
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
 

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Provider_Summary = provider_panel[(provider_panel.Organization_Name == org_name)]  
    Provider_Summary = Provider_Summary.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Provider_Summary.to_excel(writer,sheet_name='Provider Summary', index = False)
    writer.save()
    writer.close()

GPPC PCP Amherst Sheridan
GPPC PCP Buffalo Main
GPPC PCP Dunkirk Vineyard Ste 1
GPPC PCP East Aurora Main
GPPC PCP Elma Transit
GPPC PCP Hamburg Camp
GPPC PCP Hamburg Southwestern
GPPC PCP Lancaster Transit
GPPC PCP Lockport Olcott
GPPC PCP Lockport Snyder
GPPC PCP Medina Ohio
GPPC PCP Orchard Park Big Tree Ste 107


In [44]:
# Add the Organization Summary Excel Tab  
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
 

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Organization_Summary = organization_panel[(organization_panel.Organization_Name == org_name)]  
    Organization_Summary = Organization_Summary.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Organization_Summary.to_excel(writer,sheet_name='Organization Summary', index = False)
    writer.save()
    writer.close()

GPPC PCP Amherst Sheridan
GPPC PCP Buffalo Main
GPPC PCP Dunkirk Vineyard Ste 1
GPPC PCP East Aurora Main
GPPC PCP Elma Transit
GPPC PCP Hamburg Camp
GPPC PCP Hamburg Southwestern
GPPC PCP Lancaster Transit
GPPC PCP Lockport Olcott
GPPC PCP Lockport Snyder
GPPC PCP Medina Ohio
GPPC PCP Orchard Park Big Tree Ste 107


In [33]:
# Add the Organization Summary Excel Tab  
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
 

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Organization_Class_Summary = organization_class_panel
    Organization_Class_Summary = Organization_Class_Summary.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Organization_Class_Summary.to_excel(writer,sheet_name='Organization Class Summary', index = False)
    writer.save()
    writer.close()

GPPC PCP Amherst Sheridan
GPPC PCP Buffalo Main
GPPC PCP Dunkirk Vineyard Ste 1
GPPC PCP East Aurora Main
GPPC PCP Elma Transit
GPPC PCP Hamburg Camp
GPPC PCP Hamburg Southwestern
GPPC PCP Lancaster Transit
GPPC PCP Lockport Olcott
GPPC PCP Lockport Snyder
GPPC PCP Medina Ohio
GPPC PCP Orchard Park Big Tree Ste 107


In [34]:
# The following function creates patient panels based upon organization name 
def create_sheet_one(work_book):
    from openpyxl.utils import get_column_letter
    from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment    
    import openpyxl    
    
# Format the Tab      
    ws1 = work_book.active
    ws1.title = "Patient HEDIS Quality"
    ws1.sheet_properties.tabColor = CONTENT_TAB_COLOR 
     
# Format the Header as bold blue centered wordwrap    
    ws1.row_dimensions[1].height = 75   
    max_cols = ws1.max_column + 1 
    for col in range(1,max_cols):
        ws1[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws1[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws1[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)
    
# Work Sheet 1 - Set Column Widths and Header Hieght
    ws1.column_dimensions['A'].width = 15
    ws1.column_dimensions['B'].width = 15
    ws1.column_dimensions['C'].width = 120
    ws1.column_dimensions['D'].width = 10
    ws1.column_dimensions['E'].width = 10
    ws1.column_dimensions['F'].width = 10
    ws1.column_dimensions['G'].width = 12
    ws1.column_dimensions['H'].width = 30
    ws1.column_dimensions['I'].width = 30    
    ws1.column_dimensions['J'].width = 30       
    
# Replaces all underscores in header with two spaces 
    for col in range(1,max_cols):
        ws1[get_column_letter(col) + '1'].value = ws1[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws1.max_column) + str(ws1.max_row)
    ws1.auto_filter.ref = FullRange    

# Set Conditional Formatting On  
    ws1.freeze_panes = 'A2'
    ws1.conditional_formatting.add("G1:G" + str(ws1.max_row), Color_Scale_Rule )
    
    return work_book  

In [35]:
# The following function formats provider summary 
def create_sheet_two(work_book):
    from openpyxl.utils import get_column_letter
    from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment    
    import openpyxl
    
    sheets = work_book.sheetnames
    ws2 = work_book[sheets[1]]
# Format the Tab      
    ws2.title = "Provider HEDIS Quality"
    ws2.sheet_properties.tabColor = CONTENT_TAB_COLOR 
    
# Format the Header as bold blue centered wordwrap    
    ws2.row_dimensions[1].height = 75   
    max_cols = ws2.max_column + 1 
    for col in range(1,max_cols):
        ws2[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws2[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws2[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)
    
# Work Sheet 1 - Set Column Widths and Header Hieght
    ws2.column_dimensions['A'].width = 15
    ws2.column_dimensions['B'].width = 120
    ws2.column_dimensions['C'].width = 10    
    ws2.column_dimensions['D'].width = 10
    ws2.column_dimensions['E'].width = 10
    ws2.column_dimensions['F'].width = 10
    ws2.column_dimensions['G'].width = 30
    ws2.column_dimensions['H'].width = 30
    ws2.column_dimensions['I'].width = 30    
    
# Replaces all underscores in header with two spaces 
    for col in range(1,max_cols):
        ws2[get_column_letter(col) + '1'].value = ws2[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws2.max_column) + str(ws2.max_row)
    ws2.auto_filter.ref = FullRange    

# Set Conditional Formatting On  
    ws2.freeze_panes = 'A2'
    ws2.conditional_formatting.add("F1:F" + str(ws2.max_row), Color_Scale_Rule )

    return work_book  

In [36]:
def create_sheet_three(work_book):
    
    sheets = work_book.sheetnames
    ws3 = work_book[sheets[2]]
# Format the Tab      
    ws3.title = "Organization HEDIS Quality"
    ws3.sheet_properties.tabColor = CONTENT_TAB_COLOR 
  
    
# Format the Header as bold blue centered wordwrap    
    ws3.row_dimensions[1].height = 75   
    max_cols = ws3.max_column + 1 
    for col in range(1,max_cols):
        ws3[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws3[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws3[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)
    
# Work Sheet 1 - Set Column Widths and Header Hieght
    ws3.column_dimensions['A'].width = 120
    ws3.column_dimensions['B'].width = 10
    ws3.column_dimensions['C'].width = 10    
    ws3.column_dimensions['D'].width = 10
    ws3.column_dimensions['E'].width = 10
    ws3.column_dimensions['F'].width = 10
    ws3.column_dimensions['G'].width = 30
    ws3.column_dimensions['H'].width = 30
    ws3.column_dimensions['I'].width = 30    
    
# Replaces all underscores in header with two spaces 
    for col in range(1,max_cols):
        ws3[get_column_letter(col) + '1'].value = ws3[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws3.max_column) + str(ws3.max_row)
    ws3.auto_filter.ref = FullRange    

# Set Conditional Formatting On  
    ws3.freeze_panes = 'A2'
    ws3.conditional_formatting.add("E1:E" + str(ws3.max_row), Color_Scale_Rule )

    return work_book 

In [43]:
def create_sheet_four(work_book):
    
    sheets = work_book.sheetnames
    ws4 = work_book[sheets[3]]
    
    print('Sheets #4 ',sheets[3])
# Format the Tab      
    ws4.title = "Org Class HEDIS Quality"
    ws4.sheet_properties.tabColor = CONTENT_TAB_COLOR 
  
    
# Format the Header as bold blue centered wordwrap    
    ws4.row_dimensions[1].height = 75   
    max_cols = ws4.max_column + 1 
    for col in range(1,max_cols):
        ws4[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws4[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws4[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)
    
# Work Sheet 1 - Set Column Widths and Header Hieght
    ws4.column_dimensions['A'].width = 120
    ws4.column_dimensions['B'].width = 10
    ws4.column_dimensions['C'].width = 10    
    ws4.column_dimensions['D'].width = 10
    ws4.column_dimensions['E'].width = 10
    ws4.column_dimensions['F'].width = 40

    
# Replaces all underscores in header with two spaces 
    for col in range(1,max_cols):
        ws4[get_column_letter(col) + '1'].value = ws4[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws4.max_column) + str(ws4.max_row)
    ws4.auto_filter.ref = FullRange    

# Set Conditional Formatting On  
    ws4.freeze_panes = 'A2'
    ws4.conditional_formatting.add("E1:E" + str(ws4.max_row), Color_Scale_Rule )

    return work_book 


In [38]:
def create_instruction_sheet(work_book):
    
    ws_instruction  = work_book.create_sheet("Instructions")
    ws_instruction.title = "Instructions" 
    ws_instruction.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_instruction[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_instruction[get_column_letter(col) + str(row)].font = fontStyle
            ws_instruction[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
            
    for row in range(1,25):
            ws_instruction.row_dimensions[row].height = 20         
            
    ws_instruction.column_dimensions['A'].width = 3
    ws_instruction.column_dimensions['B'].width = 3
    ws_instruction.column_dimensions['C'].width = 80            
    
    
    ws_instruction.merge_cells('A1:F1')
    top_left_cell = ws_instruction['A1']
    top_left_cell.value = "Instructions for Validation"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    fontStyle = Font(size = "14", bold = "True")
    ws_instruction.merge_cells('A3:F3')
    ws_instruction['A3'].font = fontStyle
    ws_instruction['A3'] = 'Spreadsheet instructions:'

    ws_instruction['B4'] = 1
    ws_instruction['B5'] = 2 
    ws_instruction['B6'] = 3
    ws_instruction['B7'] = 4
    ws_instruction['B8'] = 5
    ws_instruction['B9'] = 6
    ws_instruction['B10'] = 7    
    ws_instruction['B11'] = 8      
    
    ws_instruction['C4'] = 'Validate Step #1 Patient Tab - Please Filter column \'Flagged for Test\' to yes... it will randomly select 25 patients'
    ws_instruction['C5'] = 'Validate Step #2 Patient Tab - Clinical Validate selected 25 patients and returned to the sender.'
    ws_instruction['C6'] = 'Validate Step #3 Patient Tab - Turn selected filter off'
    ws_instruction['C7'] = 'Validate Step #4 Patient Tab - Filter a single measure at patient panel (example Breast cancer screenihg) \' '  
    ws_instruction['C8'] = 'Validate Step #5 Patient Tab - Count qualified and completed in Excel \' ' 
    ws_instruction['C9'] = 'Validate Step #6 Provider Tab - Filter a single measure - Compare qualified and completed on selected measure to Patient counts\' '
    ws_instruction['C10'] = 'Validate Step #7 Organization Tab - Filter a single measure - Compare qualified and completed on selected measure to Provider counts\' '
    ws_instruction['C11'] = 'Validate Step #8 Organization Tab - Filter a single measure - Compare qualified and completed on selected measure to Organzation\' '     
    ws_instruction['C12'] = 'After validation is complete add or modify missing providers on the \'new provider\' tab. '   
    ws_instruction['C13'] = 'If any issues or problems occured please enter them on the \'Feedback\' tab. '       

    return work_book  


In [39]:
def create_sheet_add_physician(work_book):
    
    ws5 = work_book.create_sheet("Adding new Physicians")
    ws5.title = "Adding new Physicians" 
    ws5.sheet_properties.tabColor = USER_TAB_COLOR
    
    ws5.merge_cells('A1:G1')
    ws5.row_dimensions[1].height = 20
    top_left_cell = ws5['A1']
    top_left_cell.value = "Adding, Updating, or Removing Physicians"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws5['A2'] = 'Provider Name'
    ws5['B2'] = 'Provider NPI'
    ws5['C2'] = 'Provider Credentials'
    ws5['D2'] = 'Is Provider a Primary Care Physician PCP (Y/N) ?'
    ws5['E2'] = 'Is Provider a Specialist (Y/N) ? '
    ws5['F2'] = 'If Specialist enter Provider Specialty'
    ws5['G2'] = 'Does the Provider need to be Added or Removed or Updated (A/R/U) ? '    
    ws5['A2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['B2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['C2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['D2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['E2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['F2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['G2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    
    for col in range(1,8):
        ws5[get_column_letter(col) + '2'].font = Font(bold=True ) 
        ws5[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        ws5[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    ws5.column_dimensions['A'].width = 30
    ws5.column_dimensions['B'].width = 20
    ws5.column_dimensions['C'].width = 10
    ws5.column_dimensions['D'].width = 20
    ws5.column_dimensions['E'].width = 20
    ws5.column_dimensions['F'].width = 20   
    ws5.column_dimensions['G'].width = 20       
    
    ws5.row_dimensions[2].height = 60
 
   
    return work_book  


In [40]:
def create_sheet_feedback(work_book):
    
    # Create The Tab 
    ws_feedback = work_book.create_sheet("Feedback")
    ws_feedback.title = "Feedback or Issues" 
    ws_feedback.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_feedback[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_feedback[get_column_letter(col) + str(row)].font = fontStyle
            ws_feedback[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
    
    
    # Create The Title 
    ws_feedback.merge_cells('A1:B1')
    ws_feedback.row_dimensions[1].height = 20
    top_left_cell = ws_feedback['A1']
    top_left_cell.value = "Feedback or Issues"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws_feedback['A2'] = 'User Name'
    ws_feedback['B2'] = 'Issue or Feedback'
  
    for col in range(1,3):
        ws_feedback[get_column_letter(col) + '2'].font = Font(size = "12", bold=True ) 
        ws_feedback[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        ws_feedback[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    ws_feedback.column_dimensions['A'].width = 30
    ws_feedback.column_dimensions['B'].width = 120 

    ws_feedback.row_dimensions[2].height = 30
 
   
    return work_book  
 



In [41]:
# Automate the worksheet 
def create_work_book(work_book):
    create_sheet_one(wb)
    create_sheet_two(wb)
    create_sheet_three(wb)
    create_sheet_four(wb)    
    create_instruction_sheet(wb)
    create_sheet_add_physician(wb)  
    create_sheet_feedback(wb)      
    return work_book      

In [46]:
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))

print('Pretty .. Pretty.. Prettification') 
for x in range(len(files)): 
    print(Path_to_Save_Excel + files[x])     
    wb = load_workbook(Path_to_Excel + files[x])
    create_work_book(wb)
    wb.save(Path_to_Save_Excel + files[x]) 


Pretty .. Pretty.. Prettification
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Amherst_Sheridan.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Buffalo_Main.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Dunkirk_Vineyard_Ste_1.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_East_Aurora_Main.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Elma_Transit.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Hamburg_Camp.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Hamburg_Southwestern.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Lancaster_Transit.xlsx
Sheets #4  Organization Summary
c:/Users/josep/Documents/Kaleida/Quality_ft/GPPC_PCP_Lockport_Olcott.xlsx
Sheets #4  Organization Summary
c:/Us

In [ ]:
# Optional - Print out the compressive List of Patient Panels 
#Org_names = patient_panel.Organization_Name.unique()
#for org in Org_names:
#        print(org)

In [ ]:
# patient_panel.describe()

In [ ]:
# test examples 
#create_organization_patient_panel("GPPC PCP Springville South Cascade")
#create_organization_patient_panel("EC PCP Erie County Medical Center FM Clinic")


In [ ]:
#Provider_panel = patient_panel[(patient_panel.Provider_Name == "ADAMS, TIMOTHY")] 

In [ ]:
#   create_work_book(wb)    

In [ ]:
# patient_panel['Gender'].value_counts()

In [ ]:
# patient_panel['Organization_Name'].value_counts()

In [ ]:
# Organization_panel.head()

In [ ]:
#wb.security.workbookPassword = 'population health'
#wb.security.lockStructure = True

In [ ]:
#spread_name = '\\Users\josep\Documents\Kaleida\'+ Organization_panel
#Organization_panel.to_excel('\\Users\josep\Documents\Kaleida\GPPC PCP Springville South Cascade.xlsx')
#"C:\Users\josep\Documents\Kaleida\Validation\AA_PCP_Alan_M_Antfleck_Physician_PLLC_Delaware_Ave.xlsx"

In [ ]:
#patient_panel.rename(columns = {'patient_age_in_years':'Patient Age'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Patient Precise Age'}, inplace = True)
#patient_panel.rename(columns = {'organization_class':'Organization_Class'}, inplace = True)
#patient_panel.rename(columns = {'organization_name':'Organization_Name'}, inplace = True)
#patient_panel.rename(columns = {'provider_name':'Provider_Name'}, inplace = True)
#patient_panel.rename(columns = {'patient_name':'Patient_Name'}, inplace = True)
#patient_panel.rename(columns = {'birth_date':'Birth_Date'}, inplace = True)
#patient_panel.rename(columns = {'patient_age_in_years':'Age'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Precise_Age'}, inplace = True)
#patient_panel.rename(columns = {'gender':'Gender'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Precise Age'}, inplace = True)


In [ ]:
# Adjust Birth Date and Calculate Age  
#now = pd.Timestamp('now')
#patient_panel['Age_in_Years'] = now - patient_panel['Birth_Date']
#patient_panel['Age_in_Years'] = patient_panel['Age_in_Years']/np.timedelta64(1,'Y')
#patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date
#patient_panel.round({'Age_in_Years':1})

In [ ]:
# Trim Time from Birth Date and Add Additional fields to support User Validation 
patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date 
patient_panel['Is Patient cared for by Organization (Y/N) ?'] = ' '
patient_panel['Is Patient associated to correct PCP (Y/N) ?'] = ' '
patient_panel['Optional Comments'] = ' '
patient_panel['Flagged for Test (Y/N)'] = 'N'
patient_panel['Is Patient info Correct? (Y/N)'] = ' '

In [ ]:
# simplify and Organize the patient Panels
patient_panel = patient_panel[['Provider_Name','Patient_Name', 'Birth_Date', 'Age',
                               'Gender', 'Phone','Is Patient cared for by Organization (Y/N) ?', 
                               'Is Patient associated to correct PCP (Y/N) ?',
                               'Optional Comments','Deceased', 'Deceased_Date','Flagged for Test (Y/N)',
                               'Organization_Class', 'Organization_Name',
                               'Plan_Name', 'Payer_Name', 'Age_Years',
                               'Number_of_Conditions', 'MARA_Risk_Score',
                               'Number_of_HCC', 'Actual_CMS_RAF_Score', 'Potential_CMS_RAF_Score'
                               ,'Is Patient info Correct? (Y/N)']]